<h1>Nintendo Wii remote controlled Graphical User Interface for CrazyFlie Drone</h1> <br>
AE483: Autonomous Systems, Fall 2021<br>
Erika Jarosch, George Petrov, Justin Roskamp, Kenneth Tochihara

ReadMe <br/>
instructions for importing non-og modules <br/>
diff between mac and windows?

Import modules.

In [3]:
import numpy as np
import sympy as sym
import json
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.interpolate import interp1d

# 1. Development of Code

### 1.1 GUI.py

# 2. Run the GUI

#### Running the following box should generate the interface

In [4]:
!./run

# 3. Draw Image and Save Data 

Now connect the Nintendo Wii Remote to your computer.

The following cell contains a code snippet that details how the flight data is saved into the same directory. This can be found in the gui.py file. The "hardware_data" json file can then be loaded and parsed through in the following section.

'''
        # save flight data to directory
        directory_name = "data/" + time.strftime("%Y%m%d-%H%M%S")
        os.mkdir(directory_name)
        os.replace("src/hardware_data.json", directory_name + "/hardware_data.json")
        im = ImageGrab.grab()
        im.save(directory_name + '/gui.png')
'''

# 4. Load Data and Parse Results

#### Film a video of your test flight on a hand-held phone device, then import and save the video as "hardware_video.mov" in the same directory as this notebook.


##### Markdown

Put your video in the same directory as this notebook. Suppose this video is called `hardware_video.mov`. Then put the following code in a cell of type `Markdown` and evaluate it:
```
![some descriptive title for my video](hardware_video.mov)
```

##### HTML
Put your video in the same directory as this notebook. Suppose this video is called `hardware_video.mov`. Then put the following code in a cell of type `Code` and evaluate it:
```
%%HTML
<video width="480" controls>
    <source src="data/{INSERT_DATE_TIME}/hardware_video.mov">
</video>
```
You can change the `width` parameter to resize your video.

#### Load flight test data.

Define a function to load data from a hardware flight test and resample it at 100 Hz using linear interpolation. If only_in_flight=True, then only data for which the desired position was positive (i.e., "trying to fly" rather than "sitting on the ground") will be returned.

In [ ]:
def load_hardware_data(filename, t_min_offset=0, t_max_offset=0, only_in_flight=False):
    # load raw data
    with open(filename, 'r') as f:
        data = json.load(f)

    # convert lists to numpy arrays
    for val in data.values():
        for key in val.keys():
            val[key] = np.array(val[key])

    # create an array of times at which to subsample
    t_min = -np.inf
    t_max = np.inf
    for key, val in data.items():
        t_min = max(t_min, val['time'][0])
        t_max = min(t_max, val['time'][-1])
    t_min += t_min_offset * 1000
    t_max -= t_max_offset * 1000
    nt = int(1 + np.floor((t_max - t_min) / 10.))
    t = np.arange(0, 10 * nt, 10) / 1000.
    resampled_data = {'time': t}

    # resample raw data with linear interpolation
    for k, v in data.items():
        f = interp1d((v['time'] - t_min) / 1000., v['data'])
        resampled_data[k] = f(t)
    
    # truncate to times when o_z_des is positive
    if only_in_flight:
        i = []
        for k in ['ae483log.o_z_des', 'ctrltarget.z']:
            if k in resampled_data.keys():
                j = np.argwhere(resampled_data[k] > 0).flatten()
                if len(j) > len(i):
                    i = j
        if len(i) < 2:
            raise Exception(
                'Failed to get "only_in_flight" data.\n' + \
                ' - Did you remember to log "ae483log.o_z_des" and was it ever positive?\n' + \
                ' - Did you remember to log "ctrltarget.z" and was it ever positive?\n'
            )
        for key in resampled_data.keys():
            resampled_data[key] = resampled_data[key][i[0]:i[-1]]
        
    # return the resampled data
    return resampled_data

In [ ]:
data = load_hardware_data(
    'data/{INSERT_DATE_TIME}/hardware_data.json',  # <--- FIX ME
    t_min_offset=0.,
    t_max_offset=0.,
    only_in_flight=True,
)

Parse flight test data.

In [ ]:
# time
t = data['time']

# states
o_x_true = data['ae483log.o_x']
o_y_true = data['ae483log.o_y']
o_z_true = data['ae483log.o_z']

Define parameter values.

In [ ]:
# Acceleration of gravity
g = 9.81

# Optical flow constant (do not change)
k_flow = 0.01 * 30.0 / np.deg2rad(4.2)

# Equilibrium value of o_z
o_z_eq = 0. # <-- FIXME: replace with the equilibrium height you chose in Lab 7

# Time step (should be 0.01)
dt = t[1] - t[0]
print(f'dt = {dt:.4f} (should be 0.01)')

## 3.3 Apply observer for gain matrix chosen by equal weights

In [ ]:
def comparison_plot(t, s_hat, s_true, name, ax):
    ax.plot(t, s_hat, label=f'{name} (estimated)', linewidth=3)
    ax.plot(t, s_true, '--', label=f'{name} (true)', linewidth=2)
    ax.legend()
    ax.grid()

fig, (ax_o_x,
      ax_o_y,
      ax_o_z,
      ax_psi,
      ax_theta,
      ax_phi,
      ax_v_x,
      ax_v_y,
      ax_v_z) = plt.subplots(9, 1, figsize=(15, 25), sharex=True)
comparison_plot(t, o_x_hat, o_x_true, 'o_x', ax_o_x)
comparison_plot(t, o_y_hat, o_y_true, 'o_y', ax_o_y)
comparison_plot(t, o_z_hat, o_z_true, 'o_z', ax_o_z)
comparison_plot(t, psi_hat, psi_true, 'psi', ax_psi)
comparison_plot(t, theta_hat, theta_true, 'theta', ax_theta)
comparison_plot(t, phi_hat, phi_true, 'phi', ax_phi)
comparison_plot(t, v_x_hat, v_x_true, 'v_x', ax_v_x)
comparison_plot(t, v_y_hat, v_y_true, 'v_y', ax_v_y)
comparison_plot(t, v_z_hat, v_z_true, 'v_z', ax_v_z)
ax_v_z.set_xlabel('time (s)')
plt.show()

Plot error in each state estimate with respect to ground truth.

In [ ]:
def error_plot(t, s_err, name, ax):
    ax.plot(t, s_err, label=f'{name} (error)', linewidth=3)
    ax.legend()
    ax.grid()

fig, (ax_o_x,
      ax_o_y,
      ax_o_z,
      ax_psi,
      ax_theta,
      ax_phi,
      ax_v_x,
      ax_v_y,
      ax_v_z) = plt.subplots(9, 1, figsize=(15, 25), sharex=True)
error_plot(t, o_x_err, 'o_x', ax_o_x)
error_plot(t, o_y_err, 'o_y', ax_o_y)
error_plot(t, o_z_err, 'o_z', ax_o_z)
error_plot(t, psi_err, 'psi', ax_psi)
error_plot(t, theta_err, 'theta', ax_theta)
error_plot(t, phi_err, 'phi', ax_phi)
error_plot(t, v_x_err, 'v_x', ax_v_x)
error_plot(t, v_y_err, 'v_y', ax_v_y)
error_plot(t, v_z_err, 'v_z', ax_v_z)
ax_v_z.set_xlabel('time (s)')
plt.show()

Plot histogram of errors in each state estimate.

In [ ]:
def histogram_plot(t, s_err, name, ax):
    label = f'Error in estimate of {name} ' + \
            f'(RMSE = {np.sqrt(np.mean(s_err**2)):6.3f}, ' + \
            f'mean = {np.mean(s_err):6.3f}, ' + \
            f'std = {np.std(s_err):6.3f})'
    ax.hist(s_err, 50, label=label)
    ax.legend(fontsize=14)

fig, (ax_o_x,
      ax_o_y,
      ax_o_z,
      ax_psi,
      ax_theta,
      ax_phi,
      ax_v_x,
      ax_v_y,
      ax_v_z) = plt.subplots(9, 1, figsize=(15, 25))
histogram_plot(t, o_x_err, 'o_x', ax_o_x)
histogram_plot(t, o_y_err, 'o_y', ax_o_y)
histogram_plot(t, o_z_err, 'o_z', ax_o_z)
histogram_plot(t, psi_err, 'psi', ax_psi)
histogram_plot(t, theta_err, 'theta', ax_theta)
histogram_plot(t, phi_err, 'phi', ax_phi)
histogram_plot(t, v_x_err, 'v_x', ax_v_x)
histogram_plot(t, v_y_err, 'v_y', ax_v_y)
histogram_plot(t, v_z_err, 'v_z', ax_v_z)
fig.tight_layout()
plt.show()

## 3.5 Summary and discussion